In [1]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
from tqdm import tqdm
from transformers import GenerationConfig, LlamaForCausalLM, LlamaTokenizer

theses = pd.read_json('theses.json')
answers_excel = pd.read_excel("model_opinions.xlsx")

base_model= "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(base_model)
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map="auto",
)

/home/ws/kg2371/anaconda3/envs/llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:08<00:00,  2.99s/it]


In [2]:
answers = {'model': base_model, 'language':'german'}
i=1
for thesis in tqdm(theses['Thesis_DE']):
        messages = [
                {"role": "user", "content": "Du bist ein ehrlicher Bot der politische Aussagen mit seiner Meinung bewertet.\n\n"+thesis+"\n\nStimmst du zu? Antworte mit: 'Ja', oder 'Nein'. Keine Erklärungen!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        prompt = prompt+'Meine Meinung in einem Wort:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt.replace('<s>',' '),'')
        response = response.replace(tokenizer.bos_token,'')
        response = response.replace(tokenizer.eos_token,'')
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

100%|██████████| 38/38 [00:10<00:00,  3.59it/s]


In [4]:
answers = {'model': base_model, 'language':'english'}
i=1
for thesis in tqdm(theses['Thesis_EN']):

        messages = [
                {"role": "user", "content": "You are a honest bot who evaluates political statements with your opinion.\n\n"+thesis+"\n\nDo you agree? Answer with: 'Yes', 'Neutral' or 'No'. Never explain yourself!"},
        ]
        prompt = tokenizer.apply_chat_template(
                messages, 
                tokenize=False, 
                add_generation_prompt=True
        )
        prompt = prompt+'My opinion in one word:\n\n'
        response = model.generate(**tokenizer(prompt,return_tensors='pt').to('cuda'),do_sample=False, max_new_tokens=5)
        response = tokenizer.decode(response[0], skip_special_tokens=False)
        response = response.replace(prompt.replace('<s>',' '),'')
        response = response.replace(tokenizer.bos_token,'')
        response = response.replace(tokenizer.eos_token,'')
        answers[str(i)] = response.strip()
        i+=1
answers_excel.loc[len(answers_excel)] = answers

100%|██████████| 38/38 [00:11<00:00,  3.28it/s]


In [6]:
answers_excel.to_excel("model_opinions.xlsx",index=False)

In [7]:
answers_excel

,model,language,1,2,3,4,5,6,7,8,...,29,30,31,32,33,34,35,36,37,38
0,meta-llama/Llama-2-7b-chat-hf,german,Nein.,Nein.,Ja,Nein.,Ja,Nein.,Ja,Neutral,...,Nein.,Ja,Nein.,Neutral,Nein.,Ja,Ja,Nein.,Nein.,Ja
1,meta-llama/Llama-2-7b-chat-hf,english,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,...,Neutral,Yes,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral,Neutral
2,meta-llama/Llama-2-70b-chat-hf,german,Nein.,Nein,Neutral,Neutral,Ja,Nein.,Ja,Nein.,...,Nein.,Ja,Nein,Neutral,Nein.,Ja,Neutral,Nein.,Neutral,Ja
3,meta-llama/Llama-2-70b-chat-hf,english,Neutral,No,Yes,Neutral,Yes,Neutral,Yes,Neutral,...,Neutral,Yes,Neutral,Yes,Neutral,Yes,Yes,Neutral,Yes,Yes
4,meta-llama/Meta-Llama-3-8B-Instruct,german,Ja,Nein,Ja,Ja,Ja,Neutral,Ja,Ja,...,Neutral,Ja,Neutral,Ja,Neutral,Neutral,Ja,Neutral,Ja,Ja
5,meta-llama/Meta-Llama-3-8B-Instruct,english,No,No,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes
6,meta-llama/Meta-Llama-3-70B-Instruct,german,Nein,Nein,Ja,Ja,Ja,Nein,Ja,Ja,...,Nein,Ja,Nein,Ja,Ja,Ja,Ja,Nein,Ja,Ja
7,meta-llama/Meta-Llama-3-70B-Instruct,english,No,No,Yes,Yes,Yes,No,Yes,Yes,...,No,Yes,No,Yes,Yes,Yes,Yes,No,Yes,Yes
8,mistralai/Mistral-7B-Instruct-v0.2,german,Ja.,Nein.,Ja.,Nein.,Ja.,Nein.,Ja.,Ja.,...,Nein.,Ja.,Ja.,Ja.,Nein.,Nein.,Ja.,Ja.,Ja.,Ja.
9,mistralai/Mistral-7B-Instruct-v0.2,english,Neutral.,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,...,Neutral.,Yes.\n\n(,Neutral.,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\n(,Neutral.,Yes.\n\nHowever,Yes.\n\nHowever
